# Chapter 5: Transfer Learning
## Ex1: Pre-trained models
* Tạo 1 bộ dữ liệu Dog/Cat nhỏ (khoảng 300 hình) lấy ngẫu nhiên từ bộ dữ liệu Dog/Cat đã cung cấp ở Chapter 4
* Chọn 1 pre-trained model - VGG16 - để xây dựng model dự đoán Dog/Cat
* Lưu model vừa xây dựng xong.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/My Drive/LDS8_K275_ONLINE_NGUYENTHIKIMHOANG/Week_3/Chapter5

/content/drive/My Drive/LDS8_K275_ONLINE_NGUYENTHIKIMHOANG/Week_3/Chapter5


In [ ]:
from tensorflow.keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from tensorflow.keras import backend as k
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

## Small dataset ~ 300 images: 2 classes

### Strategy:  4

VGG16: image(224,224)

In [ ]:
img_width,img_height = 224,224
train_data_dir = 'dataset/training_set'
validation_data_dir = 'dataset/test_set'

In [ ]:
# http://www.tensorflow.org/api_docs/python/tf/keras/applications/VGG16
model = applications.VGG16(weights = 'imagenet',
                           include_top = False, #whether to include the 3 fully-connected layers at the top of the network
                           input_shape = (img_width,img_height,3))

In [ ]:
model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

## New dataset is small and similar to original dataset

In [ ]:
# so lets freeze all the VGG16 layers and train only the classifier
for layer in model.layers:
  layer.trainable = False

In [ ]:
# adding custom layers - ANN
x = model.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(1024,activation = 'relu')(x)
predictions = Dense(2, activation = 'softmax')(x)

In [ ]:
# creating the final model
model_final = Model(inputs = model.input,outputs = predictions)

In [ ]:
# compile the model
model_final.compile(loss = 'categorical_crossentropy',
                    optimizer = optimizers.SGD(lr=0.0001,momentum=0.9),
                    metrics = ['accuracy'])

In [ ]:
# Initiate the train and test generators with data Augumentation
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   horizontal_flip = True,
                                   fill_mode = 'nearest',
                                   zoom_range = 0.3,
                                   width_shift_range = 0.3,
                                   height_shift_range=0.3,
                                   rotation_range = 30)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                    target_size = (img_height,
                                                                   img_width),
                                                    batch_size = 32,
                                                    class_mode = 'categorical')

validation_generator = test_datagen.flow_from_directory(validation_data_dir,
                                                        target_size = (img_height,
                                                                       img_width),
                                                        class_mode = 'categorical')

Found 200 images belonging to 2 classes.
Found 100 images belonging to 2 classes.


In [ ]:
# save the model according to the conditions
checkpoint = ModelCheckpoint('Checkpoint_vgg16.h5',
                             monitor = 'val_loss',
                             verbose = 1,
                             save_best_only=True,
                             save_weights_only=False,
                             mode= 'auto',
                             save_freq = 1)
early = EarlyStopping(monitor = 'val_loss',
                      min_delta = 0,
                      patience = 10,
                      verbose = 1,
                      mode = 'auto')

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
callbacks = [EarlyStopping(monitor='val_loss',patience=5),ModelCheckpoint('Checkpoint_vgg16.h5',save_best_only=True)]

In [ ]:
import datetime

In [ ]:
t0 = datetime.datetime.now()
print(t0)

2022-06-01 04:21:27.715593


In [ ]:
# Train the model
history = model_final.fit(train_generator,
                steps_per_epoch = len(train_generator),
                epochs = 100,
                validation_data = validation_generator,
                validation_steps = len(validation_generator),
                verbose = 1,
                callbacks = callbacks)

Epoch 1/100
7/7 [==============================] - 5s 641ms/step - loss: 0.7776 - accuracy: 0.4950 - val_loss: 0.7752 - val_accuracy: 0.5000
Epoch 2/100
7/7 [==============================] - 4s 583ms/step - loss: 0.7879 - accuracy: 0.4800 - val_loss: 0.7459 - val_accuracy: 0.4900
Epoch 3/100
7/7 [==============================] - 4s 591ms/step - loss: 0.7339 - accuracy: 0.4700 - val_loss: 0.7102 - val_accuracy: 0.4900
Epoch 4/100
7/7 [==============================] - 4s 591ms/step - loss: 0.7140 - accuracy: 0.5600 - val_loss: 0.6831 - val_accuracy: 0.5500
Epoch 5/100
7/7 [==============================] - 4s 601ms/step - loss: 0.7180 - accuracy: 0.5450 - val_loss: 0.6667 - val_accuracy: 0.5600
Epoch 6/100
7/7 [==============================] - 4s 587ms/step - loss: 0.7119 - accuracy: 0.5550 - val_loss: 0.6539 - val_accuracy: 0.5800
Epoch 7/100
7/7 [==============================] - 4s 589ms/step - loss: 0.7008 - accuracy: 0.5450 - val_loss: 0.6432 - val_accuracy: 0.6200
Epoch 8/100
7

In [ ]:
t1 = datetime.datetime.now()
print('Thoi gian chay mo hinh: ', t1-t0)

Thoi gian chay mo hinh:  0:02:25.365233


In [ ]:
import pandas as pd

In [ ]:
history_df = pd.DataFrame(history.history)
history_df.tail()

,loss,accuracy,val_loss,val_accuracy
31,0.595738,0.695,0.541617,0.71
32,0.572380,0.685,0.539057,0.71
33,0.560795,0.705,0.475255,0.83
34,0.543088,0.715,0.514249,0.74
35,0.542864,0.725,0.485913,0.77


In [ ]:
# save model
from tensorflow.keras.models import load_model
# creates a HDF5 file 'my_model.h5'
model_final.save('dog_cat_vgg16.h5')
print('Save!!!')

Save!!!
